Update to week2 (openai) lab3 to include:
- Use of Claude ("claude-3-5-sonnet-20240620") as one of the models
- An email class for consistent structure
- Inclusion of standard footer and log of all emails sent
- I also used google to send the mails rather than sendgrid with my .env file as a lazy way to include to and from emails and my app password.

I ran this a few times to test it and I found it interesting that Claude came back a couple of times complaining about being the Head of Compliance, e.g. "As an AI assistant, I don't actually hold the position of Head of Compliance at ComplAI. I'm here to help you craft messages, not to impersonate real people or positions".  I thought this was interesting!

In [ ]:
!py -m pip install email-validator
!py -m pip install html2text

In [ ]:
#imports
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import os
from pydantic import BaseModel, EmailStr, Field
import asyncio
import smtplib
from email.mime.text import MIMEText
from datetime import date, datetime
import csv
import html2text

In [ ]:
#.env
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {openai_api_key[:13]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [ ]:
# sales agent instructions / personas
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [ ]:
# non openai models
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
ANTHROPIC_BASE_URL = "https://api.anthropic.com/v1"

gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
anthropic_client = AsyncOpenAI(base_url=ANTHROPIC_BASE_URL, api_key=anthropic_api_key)

gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)
qwen3_model = OpenAIChatCompletionsModel(model="qwen/qwen3-32b", openai_client=groq_client)
claude_model = OpenAIChatCompletionsModel(model="claude-3-5-sonnet-20240620", openai_client=anthropic_client)

In [ ]:
# map sales agents to models
sales_agent1 = Agent(name="ClaudeSales Agent",instructions=instructions1,model=claude_model)
sales_agent2 = Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3 = Agent(name="Llama3_3 Sales Agent",instructions=instructions3,model=llama3_3_model)

In [ ]:
# convert sales agents to tools
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [ ]:
# class for email structure

class EmailStructure(BaseModel):
    to: EmailStr
    from_: EmailStr = Field(alias="from")
    subject: str
    body: str
    body_html: str | None = None
    footer: str | None = None
    tags: dict[str, str] = Field(default_factory=dict)

    def generate_footer(self):
        """Generate a copyright footer with the current year."""
        year = date.today().year
        self.footer = f"<br><br>--<br>© {year} All rights reserved."

    def body_text(self):
        return html2text.html2text(self.body_html).strip()

In [ ]:
#Send email tool which includes a footer and logs the emails sent

@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send an HTML email using structured input from the EmailStructure class."""

    gmail_user = os.getenv("FROM_EMAIL") 
    gmail_app_password = os.getenv("GOOGLE_APP_PW")
    to = os.getenv("TO_EMAIL") 

    #create instance of emailstructure
    email_structure = EmailStructure(
        to=to,
        **{"from": gmail_user},
        subject=subject,
        body=html_body, 
        body_html=html_body
    )

    # Generate footer
    email_structure.generate_footer()

    # Combine HTML body with footer
    final_html_body = email_structure.body_html + (email_structure.footer or "")

    msg = MIMEText(final_html_body, "html")
    msg['Subject'] = email_structure.subject
    msg['From'] = email_structure.from_
    msg['To'] = email_structure.to

    LOG_FILE = "email_log.csv"

    def log_email(email_obj):
        file_exists = os.path.isfile(LOG_FILE)
        with open(LOG_FILE, mode="a", newline="", encoding="utf-8") as csvfile:
            writer = csv.writer(csvfile)
            if not file_exists:
                writer.writerow(["to_email", "from_email", "subject", "body", "date", "email_creator"])
            writer.writerow([
                email_structure.to,
                email_structure.from_,
                email_structure.subject,
                email_structure.body_text(),
                email_structure.tags
            ])

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(gmail_user, gmail_app_password)
            server.send_message(msg)
        print("Email sent successfully!")
        log_email(email_structure)

    except Exception as e:
        print(f"Failed to send email: {e}")


In [ ]:
#tools for email inputs
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design.  Do not include a footer as this will be added by another tool."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

email_tools = [subject_tool, html_tool, send_html_email]

In [ ]:
#emailer agent (will receive the handoff)
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
You must add a tag to the email structure to indicate the agent that created the email. \
Finally, you use the send_html_email tool to provide the structure, include a footer andsend the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [ ]:
#tool lists
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [ ]:
#input guardrail
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [ ]:
#sales manager agent initialisation

sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try but no more than 3 times. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."

careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

#message = "Send out a cold sales email addressed to Dear CEO from Alice"
message = "Send out a cold sales email addressed to Dear CEO from the head of compliance"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)